# Travaux pratiques - Classification des sols

**L’objectif** de cette deuxième séance de travaux pratiques est de vous faire manipuler des données réelles pour une tâche de classification de couverture des sols à partir d'images aériennes ou satellitaires.

Références externes utiles :
- [Documentation NumPy](https://docs.scipy.org/doc/numpy/user/index.html)  
- [Documentation SciPy](https://docs.scipy.org/doc/scipy/reference/)  
- [Documentation MatPlotLib](http://matplotlib.org/)  
- [Documentation de scikit-learn](http://scikit-learn.org/stable/index.html)  

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sklearn

## Classification pixellique

Pour commencer, nous allons considérer le jeu de données *Pavia University*. Il s'agit d'une image hyperspectrale annotée de $610\times340$ pixels, comprenant 103 bandes spectrales. L'acquisitition a été réalisée à l'aide du spectromètre ROSIS-3, et couvre une partie de l'université de Pavie en Italie. Le jeu de données total comporte ainsi 42776 pixels annotés dans 9 classes différentes : *asphalt*, *meadows*, *gravel*, *trees*, *metal sheet*, *bare soil*, *bitumen*, *brick*, et *shadow*.

In [ ]:
from scipy.io import loadmat
from urllib.request import urlretrieve
urlretrieve("https://www.ehu.eus/ccwintco/uploads/e/ee/PaviaU.mat", "PaviaU.mat")
urlretrieve("https://www.ehu.eus/ccwintco/uploads/5/50/PaviaU_gt.mat", "PaviaU_gt.mat")

In [ ]:
pavia_image = loadmat("PaviaU.mat")["paviaU"]
pavia_gt = loadmat("PaviaU_gt.mat")["paviaU_gt"]

In [ ]:
rgb_bands = (55,41,12)
pavia_rgb = pavia_image[:,:,rgb_bands]

fig = plt.figure(figsize=(12, 12))
fig.add_subplot(1,2,1)
plt.imshow((pavia_rgb - pavia_rgb.min()) / pavia_rgb.max())
plt.title("Image pseudo-RGB")
fig.add_subplot(1,2,2)
plt.imshow(pavia_gt, cmap="Pastel1")
plt.title("Vérité terrain")
plt.show()

## Question

Comment faut-il diviser le jeu de données en train/test pour que l'évaluation ne soit pas biaisée ?

Nous pouvons donc séparer le jeu de données, de façon plus ou moins arbitraire, en créant une grille approximativement carrée. Chaque bloc de la grille sera soit en train, soit en test.

In [ ]:
groups = np.zeros_like(pavia_gt)
h, w = groups.shape
idx = 0
for x in range(0, h, h // 5):
    for y in range(0, w, int(w / 2.5)):
        groups[x:x+h//5, y:y+int(w / 2.5)] = idx
        idx += 1

train_mask = (groups % 2).astype("bool")

pavia_image_train = pavia_image[train_mask]
pavia_image_test = pavia_image[~train_mask]
pavia_gt_train = pavia_gt[train_mask]
pavia_gt_test = pavia_gt[~train_mask]

fig = plt.figure(figsize=(10, 6))
fig.add_subplot(1,3,1)
plt.imshow(groups)
plt.title("Groupes")
fig.add_subplot(1,3,2)
plt.imshow(np.bitwise_and(pavia_gt, train_mask))
plt.title("Jeu d'entraînement")
fig.add_subplot(1,3,3)
plt.imshow(np.bitwise_and(pavia_gt, ~train_mask))
plt.title("Jeu d'évaluation")
plt.show()

Nous pouvons maintenant mettre en forme le jeu de données pour son traitement avec scikit-learn. L'opération dans notre cas consiste à formater les pixels (de train puis de test) sous forme d'une liste de vecteurs (chaque vecteur correspondant aux réflectances de 103 bandes spectrales). C'est donc un simple redimensionnement avec numpy. Attention toutefois, une subtilité de ce jeu de données est que la classe 0 correspond en réalité aux pixels *non annotés* : il faut donc les retirer (on ne souhaite ni s'entraîner dessus, ni s'évaluer dessus).

In [ ]:
X_train = pavia_image_train[pavia_gt_train != 0].reshape(-1, 103)
y_train = pavia_gt_train[pavia_gt_train != 0].reshape(-1)

X_test = pavia_image_test[pavia_gt_test != 0].reshape(-1, 103)
y_test = pavia_gt_test[pavia_gt_test != 0].reshape(-1)

## Question

Utilisez le classifieur de votre choix de scikit-learn et appliquez-le sur le jeu de données. Examinez ses performances avec différents paramètres *sur le jeu de validation*. N'oubliez pas de faire attention à la métrique à utiliser (le jeu de données est-il équilibré ? y a-t-il des classes plus difficiles à prédire que d'autres ?).

Il est possible ensuite d'afficher les prédictions sur le jeu de test et de les comparer à la vérité terrain :

In [ ]:
predictions = np.zeros_like(pavia_gt)
predictions[np.bitwise_and(pavia_gt != 0, ~train_mask)] = y_pred

fig = plt.figure(figsize=(10, 10))
fig.add_subplot(1,2,1)
plt.imshow(pavia_gt, cmap="Pastel1")
plt.title("Vérité terrain")
fig.add_subplot(1,2,2)
plt.imshow(predictions, cmap="Pastel1")
plt.title("Prédictions")
plt.show()

### Clustering

Dans ce cas, le jeu de données est annoté, c'est-à-dire que l'on dispose d'étiquettes de classe pour chaque observation. Cela nous permet d'entraîner des modèles *supervisés*. Toutefois, ce n'est pas toujours le cas. Si nous n'avions pas d'étiquettes, une option consiste à réaliser une classification automatique à l'aide d'une méthode de *clustering*, par exemple $k$-*means*. Dans ce cas, il n'y a pas forcément d'intérêt à séparer le jeu de données en apprentissage et validation, car nous n'avons pas de données annotées sur lesquelles calculer une vérité terrain.

Avec scikit-learn, il est possible d'utiliser l'objet [KMeans](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html) pour utiliser cet algorithme :

In [ ]:
from sklearn.cluster import KMeans

X = pavia_image.reshape(-1, 103)

kmeans = KMeans(n_clusters=10)
labels = kmeans.fit_predict(X)

## Question

À l'aide de matplotlib, affichez la carte de l'université de Pavie colorisé selon les clusters déterminés par K-Means. Comparez visuellement à la vérité terrain pour différentes valeurs de $k$. Qu'en pensez-vous ?

## Classification d'images sur EuroSAT

Pour aller plus loin, nous allons nous intéresser au jeu de
données [EuroSAT](https://github.com/phelber/EuroSAT), qui est une
base d’images contenant :

- 10 classes (bâtiments industriels, bâtiments
  résidentiels, cultures saisonnières, cultures permanentes, rivières,
  lacs & mers, végétation herbacée, routes/autoroutes, pâturages et
  forêts).  
- Il y a 270000 images au total (162000 pour l'apprentissage, 5400 pour la validation et 5400 pour l'évaluation).
- Les images sont de taille $ 64\times64 $. Les images que
  nous allons utiliser sont en couleur (RGB) mais il existe une version
  multispectrale utilisant 13 longueurs d’onde différentes.  

Pour simplifier la prise en mains, nous allons la bibliothèque [`datasets`](https://huggingface.co/docs/datasets/v1.2.0/index.html) qui permet d'importer directement ce jeu de données en une seule commande. Commençons par visualiser quelques images :

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from datasets import load_dataset
eurosat = load_dataset("blanchon/EuroSAT_RGB")

fig = plt.figure(figsize=(18, 6))
for idx, data in enumerate(eurosat['train'].shuffle()):
    if idx >= 10:
        break
    fig.add_subplot(1, 10, idx+1)
    plt.imshow(data['image'])
    plt.axis("off")
    plt.title(data['filename'].split('_')[0])
plt.show()

Il s'agit comme indiqué précédemment d'images Sentinel-2 à 10m/px de résolution. Plus précisément, il s'agit des bandes dans le domaine visible, formant une pseudo-image RGB.

La fonction `create_dataset` ci-dessous permet de convertir le jeu de données EuroSAT au format attendu par scikit-learn, c'est-à-dire des tableaux NumPy. Elle transforme le jeu de données en deux tableaux `X` (contenant les images) et `y` (content les étiquettes des classes). Les images transformées en descripteurs (*features*). Pour l'instant, on se contente de redimensionner les images au format $8\times8$ et de la transformer en vecteur. 

In [ ]:
from PIL import Image
# Exemple de redimensionnement d'image avec PIL
im = eurosat['train'][0]['image']
im.resize((8, 8))

In [ ]:
from tqdm.auto import tqdm

def create_dataset(dataset):
    X, y = [], []
    for data in tqdm(dataset):
        image, label = data['image'], data['label']
        features = np.array(image.resize((8,8))).ravel()
        X.append(features)
        y.append(label)
    return np.array(X), np.array(y)

In [ ]:
X_train, y_train = create_dataset(eurosat['train'])
X_val, y_val = create_dataset(eurosat['validation'])
X_test, y_test = create_dataset(eurosat['test'])

## Question

Utilisez le classifieur de votre choix de scikit-learn et appliquez-le sur le jeu de données. Comparez les performances de différents classifieurs avec différents paramètres *sur le jeu de validation*. N'oubliez pas de faire attention à la métrique à utiliser (le jeu de données est-il équilibré ? y a-t-il des classes plus difficiles à prédire que d'autres ?).

## Question

Répétez l'expérience précédente mais en utilisant d'autres descripteurs pour les images. Par exemple, vous pouvez utiliser les descripteurs HOG de scikit-image (`skimage.feature.hog`), d'autres facteurs de redimensionnement de l'image,  un histogramme de couleurs ou les descripteurs profonds calculés à l'aide de la fonction `extract_deep_features` ci-dessous. Quels descripteurs obtiennent les meilleurs résultats ?

In [ ]:
import torch
import torchvision
from torchvision.models import mobilenet_v3_small
from torchvision.models.mobilenetv3 import MobileNet_V3_Small_Weights

net = mobilenet_v3_small(weights=MobileNet_V3_Small_Weights.DEFAULT)
net.classifier = torch.nn.Sequential(*list(net.classifier.children())[:-3])
print(net)
def extract_deep_features(image):
    batch = MobileNet_V3_Small_Weights.DEFAULT.transforms()(image).unsqueeze(0)
    #print(batch.shape)
    #batch = torchvision.transforms.ToTensor()(image).unsqueeze(0)
    with torch.no_grad():
        return net(batch).squeeze().numpy()

extract_deep_features(eurosat['train'][0]['image']).shape

## Question

Évaluez votre meilleur modèle sur le jeu de test.